# Exploring Rule Mining with Apriori Algorithm and Retail Data
Our data source is the following: https://archive.ics.uci.edu/dataset/352/online+retail

You will need to pip install mlxtend pyopenxl

Note: pyopenxl can be slow to initally read the Excel file. Give it time.  

In [20]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 

In [21]:
# Loading the Data 
data = pd.read_excel('retail.xlsx') 
data.head() 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [22]:
data.columns # Let's explore the columns in the dataframe

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [23]:
data.Country.unique() # Let's explore the different countries represented in our dataframe

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

We will do some preliminary cleaning of our set. Namely, let's strip spaces from the description, drop entries that don't have an invoice number, and drop transactions done on credit.

In [24]:
# Stripping extra spaces in the description 
data['Description'] = data['Description'].str.strip() 
  
# Dropping the rows without any invoice number 
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
data['InvoiceNo'] = data['InvoiceNo'].astype('str') 
  
# Dropping all transactions which were done on credit 
data = data[~data['InvoiceNo'].str.contains('C')] 

Let's also split our transactions up by some countries.

In [26]:
# Transactions done in France 
basket_France = (data[data['Country'] =="France"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
  
# Transactions done in the United Kingdom 
basket_UK = (data[data['Country'] =="United Kingdom"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
  
# Transactions done in Portugal 
basket_Por = (data[data['Country'] =="Portugal"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
  
basket_Sweden = (data[data['Country'] =="Sweden"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
basket_France.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We also need to use something called [one-hot encoding](https://www.educative.io/blog/one-hot-encoding) to convert categorical columns to numbers that can work with the algorithm. We will be using map(), which we used in [Day 2 of Data Science a while ago](https://github.com/duckduckgrayduck/datascience-examples/blob/main/Day%202/Clean/clean.ipynb). 

In [27]:
# Defining the hot encoding function to make the data suitable  
# for the concerned libraries 
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
  
# Encoding the datasets 
basket_encoded = basket_France.map(hot_encode) 
basket_France = basket_encoded 
  
basket_encoded = basket_UK.map(hot_encode) 
basket_UK = basket_encoded 
  
basket_encoded = basket_Por.map(hot_encode) 
basket_Por = basket_encoded 
  
basket_encoded = basket_Sweden.map(hot_encode) 
basket_Sweden = basket_encoded 

In summary, the hot encoding is used to represent whether an individual product was present (1) or not (0) in each transaction for the respective countries.

In [28]:
basket_France.head(10)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537468,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537897,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


You can see for France, someone bought the product 10 colour space boys. 

Now we will want to build our model and interpret our results.

In [32]:
# Building the model 
frq_items = apriori(basket_France, min_support = 0.10, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

                                 antecedents                      consequents  \
40           (SET/6 RED SPOTTY PAPER PLATES)    (SET/6 RED SPOTTY PAPER CUPS)   
42  (SET/6 RED SPOTTY PAPER PLATES, POSTAGE)    (SET/6 RED SPOTTY PAPER CUPS)   
34       (STRAWBERRY LUNCH BOX WITH CUTLERY)                        (POSTAGE)   
26      (ROUND SNACK BOXES SET OF4 WOODLAND)                        (POSTAGE)   
41             (SET/6 RED SPOTTY PAPER CUPS)  (SET/6 RED SPOTTY PAPER PLATES)   

    antecedent support  consequent support   support  confidence      lift  \
40            0.127551            0.137755  0.122449    0.960000  6.968889   
42            0.107143            0.137755  0.102041    0.952381  6.913580   
34            0.122449            0.765306  0.114796    0.937500  1.225000   
26            0.158163            0.765306  0.147959    0.935484  1.222366   
41            0.137755            0.127551  0.122449    0.888889  6.968889   

    leverage  conviction  zhangs_metric  
40

/home/s/.local/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


We can see that paper plates and paper cups are often purchased together in France. This makes sense as plastic food packaging is largely banned in France due to [anti-Waste laws](https://impakter.com/new-year-new-ban-france-tackles-disposable-packaging/). 

In [43]:
frq_items2 = apriori(basket_UK, min_support = 0.03, use_colnames = True) 
rules2 = association_rules(frq_items2, metric ="lift", min_threshold = 1) 
rules2 = rules2.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules2.head()) 

/home/s/.local/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


                         antecedents                        consequents  \
3   (PINK REGENCY TEACUP AND SAUCER)  (GREEN REGENCY TEACUP AND SAUCER)   
4  (GREEN REGENCY TEACUP AND SAUCER)  (ROSES REGENCY TEACUP AND SAUCER)   
5  (ROSES REGENCY TEACUP AND SAUCER)  (GREEN REGENCY TEACUP AND SAUCER)   
8          (JUMBO BAG PINK POLKADOT)          (JUMBO BAG RED RETROSPOT)   
1       (ALARM CLOCK BAKELIKE GREEN)         (ALARM CLOCK BAKELIKE RED)   

   antecedent support  consequent support   support  confidence       lift  \
3            0.037660            0.050035  0.030910    0.820768  16.403939   
4            0.050035            0.051267  0.037553    0.750535  14.639752   
5            0.051267            0.050035  0.037553    0.732497  14.639752   
8            0.062088            0.103820  0.042053    0.677308   6.523895   
1            0.046928            0.049821  0.030160    0.642694  12.900183   

   leverage  conviction  zhangs_metric  
3  0.029026    5.300203       0.975787 

If we look at the UK's basket rules, we get some other intteresting results. People often buy teacups in sets with one green and one pink or rose colored. We can also see that if they buy a green alarm clock, they are likely to buy a red alarm clock too. This probably correlates to the Christmas season. 

In [51]:
frq_items3 = apriori(basket_Sweden, min_support = 0.10, use_colnames = True) 
rules3 = association_rules(frq_items3, metric ="lift", min_threshold = 1) 
rules3 = rules3.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules3.head()) 


                         antecedents                        consequents  \
2  (PACK OF 72 RETROSPOT CAKE CASES)   (PACK OF 60 SPACEBOY CAKE CASES)   
3   (PACK OF 60 SPACEBOY CAKE CASES)  (PACK OF 72 RETROSPOT CAKE CASES)   
0                (GUMBALL COAT RACK)                          (POSTAGE)   
4               (RABBIT NIGHT LIGHT)                          (POSTAGE)   
6    (RED TOADSTOOL LED NIGHT LIGHT)                          (POSTAGE)   

   antecedent support  consequent support   support  confidence      lift  \
2            0.111111            0.111111  0.111111         1.0  9.000000   
3            0.111111            0.111111  0.111111         1.0  9.000000   
0            0.138889            0.611111  0.138889         1.0  1.636364   
4            0.111111            0.611111  0.111111         1.0  1.636364   
6            0.138889            0.611111  0.138889         1.0  1.636364   

   leverage  conviction  zhangs_metric  
2  0.098765         inf       1.000000  
3  0

/home/s/.local/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Looking at Sweden's results with stronger correlations we see people often buy retrospot cake cases and spaceboy cake cases together. This would make sense for birthday parties, where you'd have about half of the case for girls and half for boys.